## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Verificación de la calidad de los datos

---

En esta libreta se realiza una verificación de calidad de datos con base en los planteado en el Anexo SP4. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_cv.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_cv.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Verificación de calidad de datos

**Análisis a realizar**

1. Evaluación de valores nulos (filas y columnas)
2. Evaluación de formato válido
3. Valores ajustados en rangos (ver anexos)
4. Claves únicas
5. Integridad referencial
6. Cumplimiento de reglas en valores

In [5]:
# Establecimiento de los umbrales de aceptación

FORMATEO_VALORES = 0.1
RANGOS_VALORES = 0.0
INTEGRIDAD_REF = 0.1
REGLAS_VALORES = 0.1

# Valores globales

cantidad_filas_creditos = df_creditos.shape[0]
cantidad_filas_tarjetas = df_tarjetas.shape[0]

print(f"Cantidad de filas del dataset df_creditos: {cantidad_filas_creditos}")
print(f"Cantidad de filas del dataset df_tarjetas: {cantidad_filas_tarjetas}")

Cantidad de filas del dataset df_creditos: 10127
Cantidad de filas del dataset df_tarjetas: 10127


## Dimensión: completitud

### (1a) Filas

In [6]:
# Se obtienen las cantidades de valores nulos por columna

nulos_x_columna = df_creditos.isna().sum()

print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna}")

Cantidad de filas que tienen valores nulos por atributo:
id_cliente               0
edad                     0
importe_solicitado       0
duracion_credito         0
antiguedad_empleado    337
situacion_vivienda       0
ingresos                 0
objetivo_credito         0
pct_ingreso              0
tasa_interes           912
estado_credito           0
falta_pago               0
dtype: int64


In [7]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_creditos.axes[1])

df_creditos['completitud_fila'] = (df_creditos.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_creditos[df_creditos['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print(f"Filas que incumplen el umbral de nulos en columnas [completitud_f]:")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f]:
0 (0.0)%


### (1b) Dataset

In [8]:
completitud_d = df_creditos.isnull().any(axis=1).sum()

print(f"Filas que presentan nulos en el dataset [completitud_d]:")
print(f"{completitud_d} ({round((completitud_d  / cantidad_filas_creditos) * 100, 2)})%")

Filas que presentan nulos en el dataset [completitud_d]:
1225 (12.1)%


----

## Dimensión: exactitud

### (2) Formato válido

In [ ]:
# No se encuentran atributos con formato específico

----

## Dimensión: exactitud

### (3) Valores ajustados

Atributo: **edad**

In [13]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = pd.value_counts(df_creditos['importe_solicitado']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['importe_solicitado']) - df_creditos['importe_solicitado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['edad'] > 90]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
5000     800
6000     715
8000     556
4000     446
7000     436
        ... 
750        1
700        1
19800      1
4150       1
9425       1
Name: importe_solicitado, Length: 509, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok
81,715623483.0,144,4800,3,4.0,ALQUILER,250000,INVERSIONES,0.02,13.57,0,N,0.000000,True
183,815500983.0,144,6000,2,4.0,HIPOTECA,200000,EDUCACIÓN,0.03,11.86,0,N,0.000000,True
575,717956283.0,123,20400,3,2.0,ALQUILER,80004,EDUCACIÓN,0.25,10.25,0,N,0.000000,True
747,712881483.0,123,20000,4,7.0,ALQUILER,78000,INVERSIONES,0.26,NaN,0,N,0.083333,True


Cantidad detectada: 4


In [10]:
def calcular_rangos_valores_edad():
    edad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {edad_valores_fuera_rango}")
    
    indicador = (edad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_edad()

Cantidad de filas con valores fuera de rango en atributo edad: 4
Porcentaje de filas con errores de rango de valores (atributo edad): 0.04 %
Evaluación: no cumplimiento


Se procesa el atributo: **id_cliente**

In [26]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: id_cliente

valores = pd.value_counts(df_creditos['id_cliente']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['id_cliente']) - df_creditos['id_cliente'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['id_cliente'] < 0] #Se considera que un ID de cliente tiene que ser positivo

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
713061558.0    1
719724933.0    1
806900508.0    1
794554683.0    1
718913358.0    1
              ..
716014983.0    1
712521783.0    1
780045708.0    1
719488008.0    1
716893683.0    1
Name: id_cliente, Length: 10127, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [28]:
def calcular_rangos_valores_id():
    id_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {id_valores_fuera_rango}")
    
    indicador = (id_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo id): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_id()

Cantidad de filas con valores fuera de rango en atributo edad: 0
Porcentaje de filas con errores de rango de valores (atributo id): 0.0 %
Evaluación: ok


Se procesa el atributo: **importe_solicitado**

In [30]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: importe_solicitado

valores = pd.value_counts(df_creditos['importe_solicitado']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['importe_solicitado']) - df_creditos['importe_solicitado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['importe_solicitado'] < 0] #Se considera que un importe tiene que ser positivo

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
5000     800
6000     715
8000     556
4000     446
7000     436
        ... 
750        1
700        1
19800      1
4150       1
9425       1
Name: importe_solicitado, Length: 509, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [31]:
def calcular_rangos_valores_importe():
    importe_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {importe_valores_fuera_rango}")
    
    indicador = (importe_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo importe_solicitado): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_importe()

Cantidad de filas con valores fuera de rango en atributo edad: 0
Porcentaje de filas con errores de rango de valores (atributo importe_solicitado): 0.0 %
Evaluación: ok


Se procesa el atributo **duracion_credito**

In [25]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: duracion_credito

valores = pd.value_counts(df_creditos['duracion_credito']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['duracion_credito']) - df_creditos['duracion_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[(df_creditos['duracion_credito'] < 0) & (df_creditos['duracion_credito']>72)] #Se consdiera que un credito tiene que durar más de 0 años, y que, como mucho, se lo otorgará a un cliente mayor de edad(18años) y menor de 90 años (72 años de duracion)

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
2    3404
3    3364
4    3359
Name: duracion_credito, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [32]:
def calcular_rangos_valores_duracion():
    duracion_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo duracion_credito: {duracion_valores_fuera_rango}")
    
    indicador = (duracion_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo importe_solicitado): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_importe()

Cantidad de filas con valores fuera de rango en atributo edad: 0
Porcentaje de filas con errores de rango de valores (atributo importe_solicitado): 0.0 %
Evaluación: ok


Se procesa el atributo: **antiguedad_empleado**

In [46]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: antiguedad_empleado

valores = pd.value_counts(df_creditos['antiguedad_empleado']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['antiguedad_empleado']) - df_creditos['antiguedad_empleado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[(df_creditos['antiguedad_empleado'] < 0) & (df_creditos['antiguedad_empleado']>51)] #Se consdiera que un empleado puede empezar a trabajar a los 16 y jubilarse a los 67

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
0.0      1414
2.0      1281
3.0      1128
5.0      1050
6.0      1027
1.0       993
4.0       846
7.0       784
8.0       587
9.0       414
10.0      237
11.0       27
123.0       2
Name: antiguedad_empleado, dtype: int64

Cantidad de nulos en el atributo: 337

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [47]:
def calcular_rangos_valores_antiguedad():
    antiguedad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo antiguedad_empleado: {antiguedad_valores_fuera_rango}")
    
    indicador = (antiguedad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo antiguedad_empleado): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_antiguedad()

Cantidad de filas con valores fuera de rango en atributo antiguedad_empleado: 337
Porcentaje de filas con errores de rango de valores (atributo antiguedad_empleado): 3.33 %
Evaluación: no cumplimiento


Se procesa el atributo: **ingresos**

In [44]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: ingresos

valores = pd.value_counts(df_creditos['ingresos']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['ingresos']) - df_creditos['ingresos'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[(df_creditos['ingresos'] < 0)] #

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
60000    443
30000    371
50000    337
40000    273
45000    242
        ... 
47496      1
39336      1
35900      1
24240      1
54011      1
Name: ingresos, Length: 1821, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [45]:
def calcular_rangos_valores_antiguedad():
    ingresos_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo ingresos: {ingresos_valores_fuera_rango}")
    
    indicador = (ingresos_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo ingresos): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_antiguedad()

Cantidad de filas con valores fuera de rango en atributo ingresos: 0
Porcentaje de filas con errores de rango de valores (atributo ingresos): 0.0 %
Evaluación: ok


Se procesa el atributo: **pct_ingreso**

In [49]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: pct_ingreso

valores = pd.value_counts(df_creditos['pct_ingreso']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['pct_ingreso']) - df_creditos['pct_ingreso'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[(df_creditos['pct_ingreso'] < 0) & (df_creditos['pct_ingreso'] > 1)] #

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
0.10    517
0.13    451
0.08    440
0.11    434
0.09    431
       ... 
0.67      1
0.58      1
0.72      1
0.68      1
0.70      1
Name: pct_ingreso, Length: 72, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [50]:
def calcular_rangos_valores_pctingresos():
    pcting_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo pct_ingresos: {pcting_valores_fuera_rango}")
    
    indicador = (pcting_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo pct_ingresos): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_pctingresos()

Cantidad de filas con valores fuera de rango en atributo pct_ingresos: 0
Porcentaje de filas con errores de rango de valores (atributo pct_ingresos): 0.0 %
Evaluación: ok


Se procesa el atributo: **tasa_interes**

In [51]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: tasa_interes

valores = pd.value_counts(df_creditos['tasa_interes']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['tasa_interes']) - df_creditos['tasa_interes'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[(df_creditos['tasa_interes'] < 0)] #

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
10.99    258
7.51     239
7.49     202
7.90     195
5.42     190
        ... 
13.12      1
17.88      1
17.43      1
13.93      1
17.06      1
Name: tasa_interes, Length: 306, dtype: int64

Cantidad de nulos en el atributo: 912

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [52]:
def calcular_rangos_valores_pctingresos():
    interes_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo tasa_interes: {interes_valores_fuera_rango}")
    
    indicador = (interes_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo tasa_interes): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_pctingresos()

Cantidad de filas con valores fuera de rango en atributo tasa_interes: 912
Porcentaje de filas con errores de rango de valores (atributo tasa_interes): 9.01 %
Evaluación: no cumplimiento


Se procesa el atributo **estado_credito**

In [53]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: estado_credito

valores = pd.value_counts(df_creditos['estado_credito']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['estado_credito']) - df_creditos['estado_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[(df_creditos['estado_credito'] < 0) & (df_creditos['estado_credito'] > 1)]#puede tener valor 0 o 1

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
0    7635
1    2492
Name: estado_credito, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [54]:
def calcular_rangos_valores_estado_credito():
    interes_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo tasa_interes: {interes_valores_fuera_rango}")
    
    indicador = (interes_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo tasa_interes): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_estado_credito()

Cantidad de filas con valores fuera de rango en atributo tasa_interes: 0
Porcentaje de filas con errores de rango de valores (atributo tasa_interes): 0.0 %
Evaluación: ok


Se procesa el atributo: **situacion_vivienda**


In [11]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: situacion_vivienda

valores = pd.value_counts(df_creditos['situacion_vivienda']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['situacion_vivienda']) - df_creditos['situacion_vivienda'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ALQUILER|PROPIA|HIPOTECA|OTROS' # Se define una re de los valores validos según el anexo

df_creditos['situacion_vivienda_ok'] = df_creditos['situacion_vivienda'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['situacion_vivienda_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['situacion_vivienda_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
ALQUILER    6125
HIPOTECA    3223
PROPIA       741
OTROS         38
Name: situacion_vivienda, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok


Cantidad detectada: 0


In [12]:
def calcular_rangos_valores_situacion_vivienda():
    situacion_vivienda_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: {situacion_vivienda_valores_fuera_rango}")
    
    indicador = (situacion_vivienda_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_situacion_vivienda()

Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: 0
Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): 0.0 %
Evaluación: ok


Atributo **objetivo_credito**

In [14]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: objetivo_credito

valores = pd.value_counts(df_creditos['objetivo_credito']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['objetivo_credito']) - df_creditos['objetivo_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'PERSONAL|EDUCACION|SALUD|INVERSIONES|MEJORAS_HOGAR|PAGO_DEUDAS' # Se define una re de los valores validos según el anexo

df_creditos['objetivo_credito_ok'] = df_creditos['objetivo_credito'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['objetivo_credito_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['objetivo_credito_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
EDUCACIÓN        2328
SALUD            1853
INVERSIONES      1753
PAGO_DEUDAS      1673
PERSONAL         1643
MEJORAS_HOGAR     877
Name: objetivo_credito, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,0.0,True,False
6,709106358.0,26,35000,3,8.0,ALQUILER,77100,EDUCACIÓN,0.45,12.42,1,N,0.0,True,False
13,816082233.0,26,35000,4,4.0,ALQUILER,108160,EDUCACIÓN,0.32,18.39,1,N,0.0,True,False
14,712396908.0,23,35000,4,2.0,ALQUILER,115000,EDUCACIÓN,0.30,7.90,0,N,0.0,True,False
16,709967358.0,23,35000,4,0.0,ALQUILER,120000,EDUCACIÓN,0.29,7.90,0,N,0.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10110,713924283.0,23,15000,3,2.0,HIPOTECA,65000,EDUCACIÓN,0.23,10.95,0,N,0.0,True,False
10114,712210458.0,25,9500,4,2.0,ALQUILER,57000,EDUCACIÓN,0.17,6.17,0,N,0.0,True,False
10117,710841183.0,22,9000,3,4.0,HIPOTECA,65000,EDUCACIÓN,0.14,9.63,0,N,0.0,True,False
10124,716506083.0,22,9600,2,0.0,ALQUILER,21600,EDUCACIÓN,0.44,7.49,1,N,0.0,True,False


Cantidad detectada: 2328


In [22]:
def calcular_rangos_objetivos_creditos():
    situacion_objetivo_credito_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo objetivos_creditos: {situacion_objetivo_credito_fuera_rango}")
    
    indicador = (situacion_objetivo_credito_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo objetivos_creditos): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_objetivos_creditos()

Cantidad de filas con valores fuera de rango en atributo objetivos_creditos: 0
Porcentaje de filas con errores de rango de valores (atributo objetivos_creditos): 0.0 %
Evaluación: ok


Se procesa el atributo: **falta_pago**

In [20]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: falta_pago

valores = pd.value_counts(df_creditos['falta_pago']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['falta_pago']) - df_creditos['falta_pago'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'Y|N' # Se define una re de los valores validos según el anexo

df_creditos['falta_pago_ok'] = df_creditos['falta_pago'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['falta_pago_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['falta_pago_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
N    8359
Y    1768
Name: falta_pago, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [21]:
def calcular_rangos_falta_pago():
    situacion_falta_pago_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo falta_pago: {situacion_falta_pago_fuera_rango}")
    
    indicador = (situacion_falta_pago_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo falta_pago): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_falta_pago()

Cantidad de filas con valores fuera de rango en atributo falta_pago: 0
Porcentaje de filas con errores de rango de valores (atributo falta_pago): 0.0 %
Evaluación: ok


---

## Dimensión: consistencia

### (4) Claves únicas

Dataset: **datos_creditos**

In [56]:
#Se obtiene el valor de la cantidad de filas actual
cant_antes = df_creditos.shape[0] 

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_creditos.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_creditos.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_creditos.shape[0]

# Se imprimen ambos valores
print('Dataset: creditos')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: creditos
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


Dataset: **datos_tarjetas**

In [57]:
# Se obtiene el valor de la cantidad de filas actual
cant_antes = df_tarjetas.shape[0]  

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_tarjetas.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_tarjetas.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_tarjetas.shape[0]

#Se imprimen ambos valores
print('Dataset: tarjetas')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: tarjetas
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


### (5) Integridad referencial

In [58]:
# Las uniones se hacen de a pares - revisar nombres de atributos

df_integrado = pd.merge(df_creditos, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Datos de créditos: {cantidad_filas_creditos} - Coincidencias con datos de tarjetas: {coincidencias}")

print("\nSe visualiza el dataset resultante:")
display(df_integrado.head(5))

print(f"Reporte general:\n \
- Filas del dataset creditos (inicial): {cantidad_filas_creditos}\n \
- Filas del dataset tarjetas (inicial): {cantidad_filas_tarjetas}\n \
- Errores detectados en la operación de unión: {abs(coincidencias - cantidad_filas_creditos)} \n \
- Filas del dataset unificado: {df_integrado.shape[0]}")

Datos de créditos: 10127 - Coincidencias con datos de tarjetas: 10127

Se visualiza el dataset resultante:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,45.0,SOLTERO,PASIVO,992.0,M,3421.0,DESCONOCIDO,Blue,21.0,0.0
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,POSGRADO_COMPLETO,Blue,23.0,3.0
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,Blue,82.0,2.0
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,41.0,CASADO,ACTIVO,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,Blue,59.0,2.0


Reporte general:
 - Filas del dataset creditos (inicial): 10127
 - Filas del dataset tarjetas (inicial): 10127
 - Errores detectados en la operación de unión: 0 
 - Filas del dataset unificado: 10127


In [59]:
def calcular_integridad_referencial():
    cant_problemas = cantidad_filas_creditos - df_integrado.shape[0] # Se calcula sobre el inicio (foco)
    print(f"Casos de problemas de integridad referencial: {cant_problemas}")

    indicador = (cant_problemas / cantidad_filas_creditos)
    print(f"Porcentaje de filas con problemas de integridad referencial: {round(indicador * 100, 2)} %")

    if (indicador > INTEGRIDAD_REF):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')
        
calcular_integridad_referencial()

Casos de problemas de integridad referencial: 0
Porcentaje de filas con problemas de integridad referencial: 0.0 %
Evaluación: ok


---

## Dimensión exactitud (bis)
### (6) Reglas en valores

Regla 1: Para aquellos casos en que los créditos constituyan un porcentaje de los ingresos del cliente mayor al 50% sus ingresos deberán ser mayores a 20.000.

Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos en los que sea propietario de su vivienda.

In [61]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)  

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_resultado[df_resultado.regla_pct_ingresos == 'err'].head())


# Se verifica la cantidad de elementos
aux = df_resultado[df_resultado.regla_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla: {aux.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,CASADO,ACTIVO,2244.0,M,19802.0,UNIVERSITARIO_INCOMPLETO,Blue,58.0,4.0,err
1550,711232608.0,21,8000,4,3.0,ALQUILER,15500,MEJORAS_HOGAR,0.52,7.51,...,SOLTERO,PASIVO,2235.0,F,1636.0,SECUNDARIO_COMPLETO,Blue,35.0,5.0,err
1900,711864783.0,24,10000,2,0.0,HIPOTECA,18000,PAGO_DEUDAS,0.56,12.18,...,SOLTERO,ACTIVO,1199.0,M,2201.0,UNIVERSITARIO_INCOMPLETO,Blue,32.0,1.0,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,CASADO,PASIVO,2275.0,F,1684.0,UNIVERSITARIO_INCOMPLETO,Blue,45.0,3.0,err
2809,713555133.0,22,8000,4,3.0,ALQUILER,14400,MEJORAS_HOGAR,0.56,9.99,...,DESCONOCIDO,PASIVO,2424.0,F,1667.0,SECUNDARIO_COMPLETO,Blue,40.0,3.0,err


Cantidad de filas que no cumplen la regla: 15


REGLA 2

In [66]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    duracion_credito = row.duracion_credito
    owner = row.situacion_vivienda
    
    if duracion_credito == 2 and pct_ingreso >= 0.6 and owner != "PROPIA" :
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)  

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_resultado[df_resultado.regla_pct_ingresos == 'err'].head())


# Se verifica la cantidad de elementos
aux = df_resultado[df_resultado.regla_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla: {aux.shape[0]}")

porcentaje = (aux.shape[0]/df_resultado.shape[0])*100
print(porcentaje)

Se visualizan las tuplas que no cumplen con la regla:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,CASADO,ACTIVO,2244.0,M,19802.0,UNIVERSITARIO_INCOMPLETO,Blue,58.0,4.0,err
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,...,CASADO,ACTIVO,1536.0,M,8714.0,UNIVERSITARIO_INCOMPLETO,Blue,38.0,4.0,err
1646,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,...,DIVORCIADO,PASIVO,2462.0,F,12373.0,UNIVERSITARIO_COMPLETO,Blue,40.0,2.0,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,CASADO,PASIVO,2275.0,F,1684.0,UNIVERSITARIO_INCOMPLETO,Blue,45.0,3.0,err
3041,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,...,CASADO,ACTIVO,1877.0,F,3035.0,UNIVERSITARIO_COMPLETO,Blue,37.0,3.0,err


Cantidad de filas que no cumplen la regla: 8
0.0789967413844179
